In [38]:
import itertools
import sys
%run 'ks01_Data_import.ipynb'
notebook_prefix = 'ks25'
image_counter = Counter()

### Reading precalculated in ks08 epistatic interactions

In [39]:
folder_with_precalculated_epistasis = files_dump_folder + 'epistasis_in_foursomes/'
natlib_filtering = True

#### Reading files

In [40]:
ep_dict = OrderedDict()
print '\nReading precalculated epistasis...\n%s\n' %folder_with_precalculated_epistasis
for segment in data:
    sys.stdout.flush()
    directory =  os.path.join(folder_with_precalculated_epistasis, segment.split('_')[0], '')
    list_of_files = [f for f in os.listdir(directory) if '.hdf' in f]
    for f in list_of_files:
        to_rstrip = '--epistasis_values.hdf'
        mutA, mutB = f.rstrip(to_rstrip).split('_')[-3], f.rstrip(to_rstrip).split('_')[-1]
        if natlib_filtering:
            if mutA in natural_unique_single_mutations['all_segments'] and mutB in natural_unique_single_mutations['all_segments']:
                ep_dict[mutA, mutB] = pd.read_hdf(directory + f, 'data')
        else:
            ep_dict[mutA, mutB] = pd.read_hdf(directory + f, 'data')       
print


Reading precalculated epistasis...
/Users/karen/HIS3InterspeciesEpistasis/Analysis/Karen/files_dump/epistasis_in_foursomes/




In [44]:
def get_ep_summary(epistasis_dictionary, function):
    floatprogress = FloatProgress(min=0, max=len(epistasis_dictionary))
    display(floatprogress)
    summary_dict = OrderedDict()
    for k,v in epistasis_dictionary:
        summary_dict[k] = function(v)
        floatprogress.value += 1
    return summary_dict

In [46]:
print '''
Variable "ep_dict" is a dictionary. 
Keys are tuples (mutA, mutB) and values are pandas.Series of epistasis between mutA and mutB in all backgrounds.

Function get_ep_summary takes ep_dict as an argumant and applies an arbitrary function to every pd.Series object.
For example, ep_summary = get_ep_summary(ep_dict, np.mean)

natlib_filtering is currently %s. This can be changed in the cell #2.
''' %natlib_filtering


Variable "ep_dict" is a dictionary. 
Keys are tuples (mutA, mutB) and values are pandas.Series of epistasis between mutA and mutB in all backgrounds.

Function get_ep_summary takes ep_dict as an argumant and applies an arbitrary function to every pd.Series object.
For example, ep_summary = get_ep_summary(ep_dict, np.mean)

natlib_filtering is currently True. This can be changed in the cell #2.

